In [67]:
import numpy as np
from PIL import Image, ImageEnhance
from plantcv import plantcv as pcv
from plantcv.parallel import WorkflowInputs
import cv2
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import experimental, Conv2D, MaxPooling2D, Dense, Flatten
import os
import argparse
import matplotlib.pyplot as plt

In [13]:
def loadDataset(path, img_size, batch_size):
    return tf.keras.preprocessing.image_dataset_from_directory(
        path,
        shuffle=True,
        image_size=(img_size, img_size)
    )

In [42]:
def test_corrupted_jpeg_array(img_array):
    try:
        # Tentez d'ouvrir l'image avec Pillow
        img = Image.fromarray(img_array, mode="RGB")
        img.verify()  # Vérifiez l'intégrité du fichier JPEG
        return ("OK")
    except (IOError, SyntaxError) as e:
        return (e)
    
def test_corrupted_jpeg(img):
    try:
        # Tentez d'ouvrir l'image avec Pillow
        img.verify()  # Vérifiez l'intégrité du fichier JPEG
        return ("OK")
    except (IOError, SyntaxError) as e:
        return (e)


def test_corrupted_jpeg_path(image_path):
    try:
        # Tentez d'ouvrir l'image avec Pillow
        img = Image.open(image_path)
        img.verify()  # Vérifiez l'intégrité du fichier JPEG
        print("L'image est valide.")
    except (IOError, SyntaxError) as e:
        print(f"L'image est corrompue : {e}")


In [26]:
def removeBack(img, size_fill, enhance_val, buffer_size):
    img_img = Image.fromarray(img, mode="RGB")
    contr_img = ImageEnhance.Contrast(img_img).enhance(enhance_val)
    gray_img = pcv.rgb2gray_lab(rgb_img=np.array(contr_img), channel='a')
    thresh = pcv.threshold.triangle(
        gray_img=gray_img, object_type="dark", xstep=100)
    edge_ok = pcv.fill(bin_img=thresh, size=5000)
    mask = pcv.fill(bin_img=pcv.invert(gray_img=edge_ok), size=size_fill)
    contours, _ = cv2.findContours(
        mask, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    mask_buf = mask.copy()
    if (len(contours)):
        cv2.drawContours(mask_buf,
                         contours[np.argmax([len(c) for c in contours])],
                         -1, (0, 0, 0), buffer_size)
    if ([mask_buf[0, 0], mask_buf[0, -1],
         mask_buf[0, -1], mask_buf[-1, 0]] == [0, 0, 0, 0]):
        mask_buf[0:11, 0:11] = 255
        mask_buf[-11:, -11:] = 255
        mask_buf[0:11, -11:] = 255
        mask_buf[-11:, 0:11] = 255
    mask_buf[0:1, :] = 255
    mask_buf[-1:, :] = 255
    mask_buf[:, 0:1] = 255
    mask_buf[:, -1:] = 255
    mask_buf = pcv.fill(bin_img=mask_buf, size=size_fill)
    img_modified = np.ones_like(img) * 255
    img_modified[mask_buf == 0] = img[mask_buf == 0]
    return img_modified

In [27]:
def processImgDataSet(path):
    img_path_list = [
         [[foldername, fn, '/'.join(
              [e for e in foldername.split("/") if e not in ["..", "."]])]
          for fn in filenames]
         for foldername, subdirectory, filenames in os.walk(path)
         if len(filenames)]
    img_path_list = np.array([element for sous_liste in
                              img_path_list for element in sous_liste])
    img_array = np.array(
         [np.array(Image.open(str(img_path[0] + "/" + img_path[1]), "r"))
          for img_path in img_path_list])
    img_back_removed = [removeBack(img, 5000, 1, 10) for img in img_array]
    img_back_removed_IMG = [Image.fromarray(img_array)
                            for img_array in img_back_removed]
    [os.makedirs("increased/" + path[2], exist_ok=True)
     for path in img_path_list]
    [img.save("increased/" + path[2] + "/" + path[1])
     for path, img in zip(img_path_list, img_back_removed_IMG)]
    return

In [ ]:
path = "../leaves/"
img_path_list = [
		[[foldername, fn, '/'.join(
			[e for e in foldername.split("/") if e not in ["..", "."]])]
		for fn in filenames]
		for foldername, subdirectory, filenames in os.walk(path)
		if len(filenames)]
img_path_list = np.array([element for sous_liste in
							img_path_list for element in sous_liste])
img_array = np.array(
		[np.array(Image.open(str(img_path[0] + "/" + img_path[1]), "r"))
		for img_path in img_path_list])
img_back_removed = [removeBack(img, 5000, 1, 10) for img in img_array]
img_back_removed_IMG = [Image.fromarray(img_array)
						for img_array in img_back_removed]
[os.makedirs("increased/" + path[2], exist_ok=True) for path in img_path_list]


In [ ]:
img_path_list

In [ ]:
for path, img in zip(img_path_list, img_back_removed_IMG):
	try:
		# Assurez-vous que le chemin de fichier est valide
		save_path = os.path.join("increased", path[2], path[1].split(".")[0] + ".png")
		# Enregistrez l'image avec le bon format de fichier
		img.save(save_path, format="PNG")  # Utilisez "PNG" pour les images PNG
		print(f"L'image a été enregistrée avec succès : {save_path}")
	except Exception as e:
		print(f"Erreur lors de l'enregistrement de l'image : {e}")


In [57]:
def get_dataset_partition_tf(ds, train_split=0.85,
                             shuffle=True, shuffle_size=10000):
    if shuffle:
        ds = ds.shuffle(shuffle_size, seed=12)
    len_train_dataset = int(len(ds) * train_split)
    train_dataset = ds.take(len_train_dataset)
    cv_dataset = ds.skip(len_train_dataset)
    return train_dataset, cv_dataset

In [58]:
batch_size = 32
epochs = 15
path = "../leaves"
save_dir = ""
save_name = "learnings"
img_size = 256
input_shape = (img_size, img_size, 3)

In [28]:
processImgDataSet(path)

In [76]:
path = "../leaves/"
img_path_list = [
		[[foldername, fn, '/'.join(
			[e for e in foldername.split("/") if e not in ["..", "."]]),
			foldername.split("/")[-1]]
		for fn in filenames]
		for foldername, subdirectory, filenames in os.walk(path)
		if len(filenames)]
img_path_list = np.array([element for sous_liste in
							img_path_list for element in sous_liste])
img_array = np.array(
		[np.array(Image.open(str(img_path[0] + "/" + img_path[1]), "r"))
		for img_path in img_path_list])
img_back_removed = [np.array(removeBack(img, 5000, 1, 10)) / 255.0 for img in img_array]
img_back_removed_IMG = [Image.fromarray(img_array)
                            for img_array in img_back_removed]
[os.makedirs("increased/" + path[2], exist_ok=True)
    for path in img_path_list]
[img.save(os.path.join(
    "increased", path[2], path[1].split(".")[0] + ".png"), format="PNG")
    for path, img in zip(img_path_list, img_back_removed_IMG)]


# def load_and_preprocess_image(image_path):
#     # Charger l'image
#     img = Image.open(image_path)
#     # Prétraiter l'image (redimensionner, normaliser, etc.)
#     img = np.array(img) / 255.0  # Normaliser les valeurs de pixel entre 0 et 1
#     img = tf.image.resize(img, [IMG_HEIGHT, IMG_WIDTH])  # Redimensionner l'image si nécessaire
#     return img

# # Liste des chemins vers les images et leurs étiquettes de classe correspondantes
# image_paths = ["image1.jpg", "image2.jpg", "image3.jpg", ...]
# labels = [0, 1, 2, ...]  # Liste des étiquettes de classe correspondant à chaque image

# # Paramètres de redimensionnement des images
# IMG_WIDTH, IMG_HEIGHT = 224, 224

# # Charger et prétraiter les images
# image_data = [load_and_preprocess_image(image_path) for image_path in image_paths]

# # Créer un ensemble de données TensorFlow à partir des images et de leurs étiquettes de classe
# dataset = tf.data.Dataset.from_tensor_slices((image_data, labels))

# # Afficher les premiers éléments de l'ensemble de données
# for img, label in dataset.take(5):
#     print(img.shape, label)  # A

In [59]:
dataset = loadDataset("increased/leaves/images", img_size, batch_size)
train_ds, validation_ds = get_dataset_partition_tf(dataset)
train_ds = train_ds.cache().shuffle(1000).prefetch(
    buffer_size=tf.data.AUTOTUNE)
validation_ds = validation_ds.cache().shuffle(1000).prefetch(
    buffer_size=tf.data.AUTOTUNE)
class_names = dataset.class_names

Found 7221 files belonging to 8 classes.


In [60]:
model = Sequential([
            Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
            MaxPooling2D((2, 2)),
            Conv2D(32, (3, 3), activation='relu'),
            MaxPooling2D((2, 2)),
            Flatten(),
            Dense(64, activation='relu'),
            Dense(len(class_names), activation='softmax')
        ])

In [61]:
model.build(input_shape=input_shape)
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits=False),
    optimizer='adam',
    metrics=['accuracy'])

In [ ]:
model.fit(
    train_ds,
    epochs=epochs,
    batch_size = 32,
    verbose=1,
    validation_data=validation_ds
)